# Web Scrapping of `dmarketforces.com`

## Objective
* Inspect and scrape data from `www.dmarketforces.com`
* Generate a list of blog posts, link and other available info about each post

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pprint
import pandas as pd

In [35]:
"""Scrape metadata from target URL."""
"""Create functions to scrape"""

def scrape_page_metadata(site_url, url):
    """Scrape target URL for metadata."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    pp = pprint.PrettyPrinter(indent=4)
    r = requests.get(url, headers=headers)
    r2 = requests.get(site_url, headers = headers)
    html = bs(r.content, 'html.parser')
    html2 = bs(r2.content, "html.parser")
    metadata = {
        'title': get_title(html),
        'description': get_description(html2),
        'image': get_image(html),
        'favicon': get_favicon(html, url),
        'sitename': get_site_name(html, url),
        'color': get_theme_color(html),
        'url': url
        }
    pp.pprint(metadata)
    return metadata


def get_title(html):
    """Scrape page title."""
    title = None
    if html.title.string:
        title = html.title.string
    elif html.find("meta", property="og:title"):
        title = html.find("meta", property="og:title").get('content')
    elif html.find("meta", property="twitter:title"):
        title = html.find("meta", property="twitter:title").get('content')
    elif html.find("h1"):
        title = html.find("h1").string
    return title

def get_description(html2):
    """Scrape page description."""
    description = None
    if html2.find("meta", property="title"):
        description = html2.find("meta", property="title").get('content')
    elif html2.find("meta", property="og:title"):
        description = html2.find("meta", property="og:title").get('content')
    elif html2.find("meta", property="twitter:title"):
        description = html2.find("meta", property="twitter:title").get('content')
    elif html2.find("p"):
        description = html2.find("p").contents
    return description


def get_image(html):
    """Scrape share image."""
    image = None
    if html.find("meta", property="image"):
        image = html.find("meta", property="image").get('content')
    elif html.find("meta", property="og:image"):
        image = html.find("meta", property="og:image").get('content')
    elif html.find("meta", property="twitter:image"):
        image = html.find("meta", property="twitter:image").get('content')
    elif html.find("img", src=True):
        image = html.find_all("img").get('src')
    return image


def get_site_name(html, url):
    """Scrape site name."""
    if html.find("meta", property="og:site_name"):
        site_name = html.find("meta", property="og:site_name").get('content')
    elif html.find("meta", property='twitter:title'):
        site_name = html.find("meta", property="twitter:title").get('content')
    else:
        site_name = url.split('//')[1]
        return site_name.split('/')[0].rsplit('.')[1].capitalize()
    return site_name


def get_favicon(html, url):
    """Scrape favicon."""
    if html.find("link", attrs={"rel": "icon"}):
        favicon = html.find("link", attrs={"rel": "icon"}).get('href')
    elif html.find("link", attrs={"rel": "shortcut icon"}):
        favicon = html.find("link", attrs={"rel": "shortcut icon"}).get('href')
    else:
        favicon = f'{url.rstrip("/")}/favicon.ico'
    return favicon


def get_theme_color(html):
    """Scrape brand color."""
    if html.find("meta", property="theme-color"):
        color = html.find("meta", property="theme-color").get('content')
        return color
    return None


In [176]:
from bs4 import BeautifulSoup as bs
# Query website using website url
def query_site(site_url:str):
    response  = requests.get(site_url)
    
    #beautify content to make it readable
    soup = bs(response.content, "html.parser")

    # inspect and Find all class of interest 
    site_content = (soup.find("ul", class_= "td-mobile-main-menu")).find_all("a")
    
    #create list of main navigation links in site
    global links
    links = []
    for item in site_content:
        link = item.attrs["href"]
        links.append(link)
    return links

In [177]:
query_site("https://dmarketforces.com/")

['https://dmarketforces.com/',
 'https://dmarketforces.com/category/news/',
 'https://dmarketforces.com/category/economy-news-archives/',
 'https://dmarketforces.com/category/analysis/',
 'https://dmarketforces.com/category/world-news/business/',
 'https://dmarketforces.com/category/world-news/industries/',
 'https://dmarketforces.com/category/news/healthcare/',
 'https://dmarketforces.com/category/investments/',
 'https://dmarketforces.com/category/featured/']

### News 

In [178]:
# Call website (link)
site_link = "https://dmarketforces.com/category/news/"
response  = requests.get(site_link)
response.status_code

200

In [179]:
#beautify content to make it readable
soup = bs(response.content, "html.parser")
#print(soup.prettify())

# inspect and Find all class of interest 
site_content = soup.find_all("a", class_= "td-image-wrap")
#site_content #td-image-wrap
    
# Collect all links in site content in a list
links = []
for item in site_content:
    link = item.attrs["href"]
    links.append(link)
#print(links)

# Get info for each link in Link List
dat = []
for link in links:
    lis = scrape_page_metadata(site_link, link)
    dat.append(lis)
#print(lis)
    


{   'color': None,
    'description': 'News Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/09/Investment-Returns-Swerve-in-Nigerias-Equity-Fixed-Income-Market.png',
    'sitename': 'MarketForces Africa',
    'title': "Investment Returns Swerve in Nigeria's Equity, Fixed Income...",
    'url': 'https://dmarketforces.com/investment-returns-swerve-in-nigerias-equity-fixed-income-market/'}
{   'color': None,
    'description': 'News Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/09/Oil-Prices-Jump-as-OPEC-Meet-to-Discuss-Output-Raise.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Oil Prices Jump as OPEC+ Meet to Discuss Output Raise',
    'url': 'https://dmarketforces.com/oil-prices-jump-as-opec-meet-

In [180]:
#News table
news_table = pd.DataFrame(dat)
news_table.drop(["color", "favicon"], axis = 1)
news_table = news_table.reindex(columns = ["sitename",  "description", "title", "url", "image"])
news_table

,sitename,description,title,url,image
0,MarketForces Africa,News Archives,"Investment Returns Swerve in Nigeria's Equity,...",https://dmarketforces.com/investment-returns-s...,https://dmarketforces.com/wp-content/uploads/2...
1,MarketForces Africa,News Archives,Oil Prices Jump as OPEC+ Meet to Discuss Outpu...,https://dmarketforces.com/oil-prices-jump-as-o...,https://dmarketforces.com/wp-content/uploads/2...
2,MarketForces Africa,News Archives,Federal Government Disburses N56 Billion for S...,https://dmarketforces.com/federal-government-d...,https://dmarketforces.com/wp-content/uploads/2...
3,MarketForces Africa,News Archives,Nigerian Exchange Sinks as Investors Sustain S...,https://dmarketforces.com/nigerian-exchange-si...,https://dmarketforces.com/wp-content/uploads/2...
4,MarketForces Africa,News Archives,"Federal Govt. Plans to Boost Oil, Gas Reserves...",https://dmarketforces.com/federal-govt-plans-t...,https://dmarketforces.com/wp-content/uploads/2...
5,MarketForces Africa,News Archives,Nigeria Records Biggest Economic Growth in 6 Y...,https://dmarketforces.com/nigeria-records-bigg...,https://dmarketforces.com/wp-content/uploads/2...
6,MarketForces Africa,News Archives,MTN Nigeria Dodges Alleged $2 Billion Indebted...,https://dmarketforces.com/mtn-nigeria-dodges-a...,https://dmarketforces.com/wp-content/uploads/2...
7,MarketForces Africa,News Archives,CBN Announces Bitt Inc. as Technical Partner f...,https://dmarketforces.com/cbn-announces-bitt-i...,https://dmarketforces.com/wp-content/uploads/2...
8,MarketForces Africa,News Archives,T-Bills Yield Falls as Market Erases Premium o...,https://dmarketforces.com/t-bills-yield-falls-...,https://dmarketforces.com/wp-content/uploads/2...
9,MarketForces Africa,News Archives,U.S Dollar Trades Under Pressure over Fed's Ta...,https://dmarketforces.com/u-s-dollar-trades-un...,https://dmarketforces.com/wp-content/uploads/2...


### Economy News

In [181]:
# Call website (link)
site_link = "https://dmarketforces.com/category/economy-news-archives/"
response  = requests.get(site_link)
response.status_code

#beautify content to make it readable
soup = bs(response.content, "html.parser")
#print(soup.prettify())

# inspect and Find all class of interest 
site_content = soup.find_all("a", class_= "td-image-wrap")
#site_content #td-image-wrap
    
# Collect all links in site content in a list
links = []
for item in site_content:
    link = item.attrs["href"]
    links.append(link)
#print(links)

# Get info for each link in Link List
dat = []
for link in links:
    lis = scrape_page_metadata(site_link, link)
    dat.append(lis)
#print(lis)
    


{   'color': None,
    'description': 'Economy Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/FG-Gets-Over-80-of-FX-Earnings-from-Oil-Gas-Exports.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'FG Gets Over 80% of FX Earnings from Oil, Gas Exports',
    'url': 'https://dmarketforces.com/fg-gets-over-80-of-fx-earnings-from-oil-gas-exports/'}
{   'color': None,
    'description': 'Economy Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Nigerias-GDP-in-Recovery-Mode-Analysts-See-3.1-Growth.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Nigeria’s GDP in Recovery Mode, Analysts See 3.1% Growth',
    'url': 'https://dmarketforces.com/nigerias-gdp-in-recovery-mode-analysts-see-3-1-growth/

{   'color': None,
    'description': 'Economy Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/01/President-Muhammadu-Buhari-2.jpg',
    'sitename': 'MarketForces Africa',
    'title': "Growth-starved Nigeria's Economy Expends 83% of Revenue to "
             'Service Debts',
    'url': 'https://dmarketforces.com/growth-starved-nigerias-economy-expends-83-of-revenue-to-service-debts/'}
{   'color': None,
    'description': 'Economy Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}
{   '

{   'color': None,
    'description': 'Economy Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/07/How-SAP-Helps-Resolve-Capital-Issues-for-MSMEs.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'How SAP Helps Resolve Capital Issues for MSMEs',
    'url': 'https://dmarketforces.com/how-sap-helps-resolve-capital-issues-for-msmes/'}
{   'color': None,
    'description': 'Economy Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}
{   'color': None,
    'description': 'Economy Archiv

In [182]:
#Format Economy News table
econews_table = pd.DataFrame(dat)
econews_table.drop(["color", "favicon"], axis = 1)
econews_table = econews_table.reindex(columns = ["sitename", "description", "title", "url", "image"])
econews_table

,sitename,description,title,url,image
0,MarketForces Africa,Economy Archives,"FG Gets Over 80% of FX Earnings from Oil, Gas ...",https://dmarketforces.com/fg-gets-over-80-of-f...,https://dmarketforces.com/wp-content/uploads/2...
1,MarketForces Africa,Economy Archives,"Nigeria’s GDP in Recovery Mode, Analysts See 3...",https://dmarketforces.com/nigerias-gdp-in-reco...,https://dmarketforces.com/wp-content/uploads/2...
2,MarketForces Africa,Economy Archives,Nigeria’s GDP Expands 5.1% in the Second Quarter,https://dmarketforces.com/nigerias-gdp-expands...,https://dmarketforces.com/wp-content/uploads/2...
3,MarketForces Africa,Economy Archives,"Eurobond, SDR Will Boost Nigeria's Foreign Res...",https://dmarketforces.com/eurobond-sdr-will-bo...,https://dmarketforces.com/wp-content/uploads/2...
4,MarketForces Africa,Economy Archives,Domestic Borrowings to Exceed N1.3 Trillion in...,https://dmarketforces.com/domestic-borrowings-...,https://dmarketforces.com/wp-content/uploads/2...
5,MarketForces Africa,Economy Archives,Nigeria Spends 98% of Total Income to Service ...,https://dmarketforces.com/nigeria-spends-98-of...,https://dmarketforces.com/wp-content/uploads/2...
6,MarketForces Africa,Economy Archives,Dollar Inflow Drops 54% as Foreign Investors D...,https://dmarketforces.com/dollar-inflow-drops-...,https://dmarketforces.com/wp-content/uploads/2...
7,MarketForces Africa,Economy Archives,MPC Will Keep Policy Rates to Drive GDP Reboun...,https://dmarketforces.com/mpc-will-keep-policy...,https://dmarketforces.com/wp-content/uploads/2...
8,MarketForces Africa,Economy Archives,Federal Government Borrows Large from Treasury...,https://dmarketforces.com/federal-government-b...,https://dmarketforces.com/wp-content/uploads/2...
9,MarketForces Africa,Economy Archives,Pension Assets Expand to N12.40 Trillion as PF...,https://dmarketforces.com/pension-assets-expan...,https://dmarketforces.com/wp-content/uploads/2...


### Analysis

In [183]:
# Call website (link)
site_link = "https://dmarketforces.com/category/analysis/"
response  = requests.get(site_link)
response.status_code

#beautify content to make it readable
soup = bs(response.content, "html.parser")
#print(soup.prettify())

# inspect and Find all class of interest 
site_content = soup.find_all("a", class_= "td-image-wrap")
#site_content #td-image-wrap
    
# Collect all links in site content in a list
links = []
for item in site_content:
    link = item.attrs["href"]
    links.append(link)
#print(links)

# Get info for each link in Link List
dat = []
for link in links:
    lis = scrape_page_metadata(site_link, link)
    dat.append(lis)
#print(lis)
    

{   'color': None,
    'description': 'Analysis Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Rising-Costs-Eclipsed-Nestle-Nigeria-Tenacity-to-Boost-Earnings.jpeg',
    'sitename': 'MarketForces Africa',
    'title': 'Rising Costs Eclipsed Nestlé Nigeria Tenacity to Boost Earnings',
    'url': 'https://dmarketforces.com/rising-costs-eclipsed-nestle-nigeria-tenacity-to-boost-earnings/'}
{   'color': None,
    'description': 'Analysis Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Dangote-Sugar-Could-Offer-Investors-Inflation-Protected-Return-Analysts.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Dangote Sugar Could Offer Investors Inflation-Protected Return '
             '-Analysts',
    '

{   'color': None,
    'description': 'Analysis Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/01/President-Muhammadu-Buhari-2.jpg',
    'sitename': 'MarketForces Africa',
    'title': "Growth-starved Nigeria's Economy Expends 83% of Revenue to "
             'Service Debts',
    'url': 'https://dmarketforces.com/growth-starved-nigerias-economy-expends-83-of-revenue-to-service-debts/'}
{   'color': None,
    'description': 'Analysis Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}
{  

{   'color': None,
    'description': 'Analysis Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/07/How-SAP-Helps-Resolve-Capital-Issues-for-MSMEs.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'How SAP Helps Resolve Capital Issues for MSMEs',
    'url': 'https://dmarketforces.com/how-sap-helps-resolve-capital-issues-for-msmes/'}
{   'color': None,
    'description': 'Analysis Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}
{   'color': None,
    'description': 'Analysis Arc

In [184]:
#Format Analysis table
analy_table = pd.DataFrame(dat)
analy_table.drop(["color", "favicon"], axis = 1)
analy_table = analy_table.reindex(columns = ["sitename", "description", "title", "url", "image"])
analy_table

,sitename,description,title,url,image
0,MarketForces Africa,Analysis Archives,Rising Costs Eclipsed Nestlé Nigeria Tenacity ...,https://dmarketforces.com/rising-costs-eclipse...,https://dmarketforces.com/wp-content/uploads/2...
1,MarketForces Africa,Analysis Archives,Dangote Sugar Could Offer Investors Inflation-...,https://dmarketforces.com/dangote-sugar-could-...,https://dmarketforces.com/wp-content/uploads/2...
2,MarketForces Africa,Analysis Archives,NB Plc: Valuation Spots Weak Upside as Cost Pr...,https://dmarketforces.com/nb-plc-valuation-spo...,https://dmarketforces.com/wp-content/uploads/2...
3,MarketForces Africa,Analysis Archives,Okomu Oil: Analysts Project 20% Upside For Sto...,https://dmarketforces.com/okomu-oil-analysts-p...,https://dmarketforces.com/wp-content/uploads/2...
4,MarketForces Africa,Analysis Archives,Flour Mills of Nigeria CEO Seeks to Rally Shar...,https://dmarketforces.com/flour-mills-of-niger...,https://dmarketforces.com/wp-content/uploads/2...
5,MarketForces Africa,Analysis Archives,Dangote Cement: Analysts Forecast 13% Increase...,https://dmarketforces.com/dangote-cement-analy...,https://dmarketforces.com/wp-content/uploads/2...
6,MarketForces Africa,Analysis Archives,UACN: Analysts Rate Stock Hold After Q2 Earnings,https://dmarketforces.com/uacn-analysts-rate-s...,https://dmarketforces.com/wp-content/uploads/2...
7,MarketForces Africa,Analysis Archives,"Lafarge Africa: Attractive Opportunity, Analys...",https://dmarketforces.com/lafarge-africa-attra...,https://dmarketforces.com/wp-content/uploads/2...
8,MarketForces Africa,Analysis Archives,Sterling Bank Extends 58% of Its Loan Portfoli...,https://dmarketforces.com/sterling-bank-extend...,https://dmarketforces.com/wp-content/uploads/2...
9,MarketForces Africa,Analysis Archives,"Ecobank Earnings for 2021 to Hit 7-Year High, ...",https://dmarketforces.com/ecobank-earnings-for...,https://dmarketforces.com/wp-content/uploads/2...


### World Business News

In [185]:
# Call website (link)
site_link = "https://dmarketforces.com/category/world-news/business/"
response  = requests.get(site_link)
response.status_code

#beautify content to make it readable
soup = bs(response.content, "html.parser")
#print(soup.prettify())

# inspect and Find all class of interest 
site_content = soup.find_all("a", class_= "td-image-wrap")
#site_content #td-image-wrap
    
# Collect all links in site content in a list
links = []
for item in site_content:
    link = item.attrs["href"]
    links.append(link)
#print(links)

# Get info for each link in Link List
dat = []
for link in links:
    lis = scrape_page_metadata(site_link, link)
    dat.append(lis)
#print(lis)
    

{   'color': None,
    'description': 'Business Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/04/Rates-on-Debt-Instruments-Surge-as-Liquidity-Squeeze-Persists.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Rates on Debt Instruments Surge as Liquidity Squeeze Persists',
    'url': 'https://dmarketforces.com/rates-on-debt-instruments-surge-as-liquidity-squeeze-persists/'}
{   'color': None,
    'description': 'Business Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/04/Nigerian-Exchange-Rally-as-Investors-Gain-N59-Billion.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Nigerian Exchange Rally as Investors Gain N59 Billion',
    'url': 'https://dmarketforces.com/nigerian-exchange-rally-as-

{   'color': None,
    'description': 'Business Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/01/President-Muhammadu-Buhari-2.jpg',
    'sitename': 'MarketForces Africa',
    'title': "Growth-starved Nigeria's Economy Expends 83% of Revenue to "
             'Service Debts',
    'url': 'https://dmarketforces.com/growth-starved-nigerias-economy-expends-83-of-revenue-to-service-debts/'}
{   'color': None,
    'description': 'Business Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}
{  

{   'color': None,
    'description': 'Business Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/07/How-SAP-Helps-Resolve-Capital-Issues-for-MSMEs.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'How SAP Helps Resolve Capital Issues for MSMEs',
    'url': 'https://dmarketforces.com/how-sap-helps-resolve-capital-issues-for-msmes/'}
{   'color': None,
    'description': 'Business Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}
{   'color': None,
    'description': 'Business Arc

In [189]:
#Format World Business News table
wb_table = pd.DataFrame(dat)
wb_table.drop(["color", "favicon"], axis = 1)
wb_table = wb_table.reindex(columns = ["sitename", "description", "title", "url", "image"])
wb_table

,sitename,description,title,url,image
0,MarketForces Africa,Business Archives,Rates on Debt Instruments Surge as Liquidity S...,https://dmarketforces.com/rates-on-debt-instru...,https://dmarketforces.com/wp-content/uploads/2...
1,MarketForces Africa,Business Archives,Nigerian Exchange Rally as Investors Gain N59 ...,https://dmarketforces.com/nigerian-exchange-ra...,https://dmarketforces.com/wp-content/uploads/2...
2,MarketForces Africa,Business Archives,NGX Gains N20.9 Billion on Improved Investors ...,https://dmarketforces.com/ngx-gains-n20-9-bill...,https://dmarketforces.com/wp-content/uploads/2...
3,MarketForces Africa,Business Archives,Debt Market Trades Cautious over Disappointing...,https://dmarketforces.com/debt-market-trades-c...,https://dmarketforces.com/wp-content/uploads/2...
4,MarketForces Africa,Business Archives,Weak Liquidity Keeps Interbank Rates at Double...,https://dmarketforces.com/weak-liquidity-keeps...,https://dmarketforces.com/wp-content/uploads/2...
5,MarketForces Africa,Business Archives,Stocks Appreciate N3.9Bn despite Waning Invest...,https://dmarketforces.com/stocks-appreciate-n3...,https://dmarketforces.com/wp-content/uploads/2...
6,MarketForces Africa,Business Archives,"OPEC+ Raised Output By 1,141Kb/d over 2-Month",https://dmarketforces.com/opec-raised-output-b...,https://dmarketforces.com/wp-content/uploads/2...
7,MarketForces Africa,Business Archives,Stocks Tumble in Volatile Trade as Investors N...,https://dmarketforces.com/stocks-tumble-in-vol...,https://dmarketforces.com/wp-content/uploads/2...
8,MarketForces Africa,Business Archives,5 African Startups Compete for $500K in Emergi...,https://dmarketforces.com/5-african-startups-c...,https://dmarketforces.com/wp-content/uploads/2...
9,MarketForces Africa,Business Archives,Nigerian T-Bills Discount Rates Closed Flat as...,https://dmarketforces.com/nigerian-t-bills-dis...,https://dmarketforces.com/wp-content/uploads/2...


### World Industries News

In [190]:
# Call website (link)
site_link = "https://dmarketforces.com/category/world-news/industries/"
response  = requests.get(site_link)
response.status_code

#beautify content to make it readable
soup = bs(response.content, "html.parser")
#print(soup.prettify())

# inspect and Find all class of interest 
site_content = soup.find_all("a", class_= "td-image-wrap")
#site_content #td-image-wrap
    
# Collect all links in site content in a list
links = []
for item in site_content:
    link = item.attrs["href"]
    links.append(link)
#print(links)

# Get info for each link in Link List
dat = []
for link in links:
    lis = scrape_page_metadata(site_link, link)
    dat.append(lis)
#print(lis)
    

{   'color': None,
    'description': 'Industries Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/07/Grifols-to-Build-Prod.-Facility-for-Intravenous-Solutions-in-Nigeria.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Grifols to Build Prod. Facility for Intravenous Solutions in '
             'Nigeria',
    'url': 'https://dmarketforces.com/grifols-to-build-prod-facility-for-intravenous-solutions-in-nigeria/'}
{   'color': None,
    'description': 'Industries Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/07/BUA-Seeks-Reduce-Influence-of-Cement-Oligarchs-to-Crash-Price.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'BUA Seeks Reduce Influence of Cement Oligarchs to Crash Price',
    'url'

{   'color': None,
    'description': 'Industries Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/01/President-Muhammadu-Buhari-2.jpg',
    'sitename': 'MarketForces Africa',
    'title': "Growth-starved Nigeria's Economy Expends 83% of Revenue to "
             'Service Debts',
    'url': 'https://dmarketforces.com/growth-starved-nigerias-economy-expends-83-of-revenue-to-service-debts/'}
{   'color': None,
    'description': 'Industries Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}

{   'color': None,
    'description': 'Industries Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/07/How-SAP-Helps-Resolve-Capital-Issues-for-MSMEs.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'How SAP Helps Resolve Capital Issues for MSMEs',
    'url': 'https://dmarketforces.com/how-sap-helps-resolve-capital-issues-for-msmes/'}
{   'color': None,
    'description': 'Industries Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}
{   'color': None,
    'description': 'Industri

In [192]:
#Format World Industries News table
wi_table = pd.DataFrame(dat)
wi_table.drop(["color", "favicon"], axis = 1)
wi_table = wi_table.reindex(columns = ["sitename", "description", "title", "url", "image"])
wi_table

,sitename,description,title,url,image
0,MarketForces Africa,Industries Archives,Grifols to Build Prod. Facility for Intravenou...,https://dmarketforces.com/grifols-to-build-pro...,https://dmarketforces.com/wp-content/uploads/2...
1,MarketForces Africa,Industries Archives,BUA Seeks Reduce Influence of Cement Oligarchs...,https://dmarketforces.com/bua-seeks-reduce-inf...,https://dmarketforces.com/wp-content/uploads/2...
2,MarketForces Africa,Industries Archives,EDP Funds Seven Solar Energy Projects in Niger...,https://dmarketforces.com/edp-funds-seven-sola...,https://dmarketforces.com/wp-content/uploads/2...
3,MarketForces Africa,Industries Archives,"Textile, Fashion World's Second-Largest Indust...",https://dmarketforces.com/textile-fashion-worl...,https://dmarketforces.com/wp-content/uploads/2...
4,MarketForces Africa,Industries Archives,"Cement: Factory Price Per Bag is N2,450 – Dangote",https://dmarketforces.com/cement-factory-price...,https://dmarketforces.com/wp-content/uploads/2...
5,MarketForces Africa,Industries Archives,Nigerian Cement Oligarchs Record N1.2 Trillion...,https://dmarketforces.com/nigerian-cement-olig...,https://dmarketforces.com/wp-content/uploads/2...
6,MarketForces Africa,Industries Archives,"Dangote Cement Debunks Price Hike, Says Middle...",https://dmarketforces.com/dangote-cement-debun...,https://dmarketforces.com/wp-content/uploads/2...
7,MarketForces Africa,Industries Archives,Tesla’s Share Price Down 30% after $1.5Bn Bitc...,https://dmarketforces.com/teslas-share-price-d...,https://dmarketforces.com/wp-content/uploads/2...
8,MarketForces Africa,Industries Archives,Naira Struggles Unsuccessfully to Retain Value...,https://dmarketforces.com/naira-struggles-unsu...,https://dmarketforces.com/wp-content/uploads/2...
9,MarketForces Africa,Industries Archives,"AGPC Raises US$260m to complete ANOH Project, ...",https://dmarketforces.com/agpc-raises-us260m-t...,https://dmarketforces.com/wp-content/uploads/2...


### Healthcare News

In [196]:
# Call website (link)
site_link = "https://dmarketforces.com/category/news/healthcare/"
response  = requests.get(site_link)
response.status_code

#beautify content to make it readable
soup = bs(response.content, "html.parser")
#print(soup.prettify())

# inspect and Find all class of interest 
site_content = soup.find_all("a", class_= "td-image-wrap")
#site_content #td-image-wrap
    
# Collect all links in site content in a list
links = []
for item in site_content:
    link = item.attrs["href"]
    links.append(link)
#print(links)

# Get info for each link in Link List
dat = []
for link in links:
    lis = scrape_page_metadata(site_link, link)
    dat.append(lis)
#print(lis)
    

{   'color': None,
    'description': 'Healthcare Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/06/World-Leaders-Unite-on-Equitable-Access-for-COVID-19-Vaccines.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'World Leaders Unite on Equitable Access for COVID-19 Vaccines',
    'url': 'https://dmarketforces.com/world-leaders-unite-on-equitable-access-for-covid-19-vaccines/'}
{   'color': None,
    'description': 'Healthcare Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/03/NCDC-Records-149882-COVID-19-Patients-Recoveries.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'NCDC Records 149,882 COVID-19 Patients Recoveries',
    'url': 'https://dmarketforces.com/ncdc-records-149882-covid-19-pat

{   'color': None,
    'description': 'Healthcare Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}
{   'color': None,
    'description': 'Healthcare Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/09/Investment-Returns-Swerve-in-Nigerias-Equity-Fixed-Income-Market.png',
    'sitename': 'MarketForces Africa',
    'title': "Investment Returns Swerve in Nigeria's Equity, Fixed Income...",
    'url': 'https://dmarketforces.com/investment-returns-swerve-in-nigerias-equity-fixed-income-marke

{   'color': None,
    'description': 'Healthcare Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2020/05/Enterpreneur-e1619167255352.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'The Reality and Fantasy of Being an Entrepreneur',
    'url': 'https://dmarketforces.com/the-reality-and-fantasy-of-being-an-entrepreneur/'}
{   'color': None,
    'description': 'Healthcare Archives',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/02/Who-Wants-to-Raise-Capital-for-Start-Up-Small-Business.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Who Wants to Raise Capital for Start-Up, Small Business?',
    'url': 'https://dmarketforces.com/who-wants-to-raise-capital-for-start-up-small-business/'}
{   'color': None,
  

In [197]:
#Format Healthcare News table
hc_table = pd.DataFrame(dat)
hc_table.drop(["color", "favicon"], axis = 1)
hc_table = hc.reindex(columns = ["sitename", "description", "title", "url", "image"])
hc

NameError: name 'hc' is not defined

### Investment News

In [193]:
# Call website (link)
site_link = "https://dmarketforces.com/category/investment/"
response  = requests.get(site_link)
response.status_code

#beautify content to make it readable
soup = bs(response.content, "html.parser")
#print(soup.prettify())

# inspect and Find all class of interest 
site_content = soup.find_all("a", class_= "td-image-wrap")
#site_content #td-image-wrap
    
# Collect all links in site content in a list
links = []
for item in site_content:
    link = item.attrs["href"]
    links.append(link)
#print(links)

# Get info for each link in Link List
dat = []
for link in links:
    lis = scrape_page_metadata(site_link, link)
    dat.append(lis)
#print(lis)
    

{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/09/Investment-Returns-Swerve-in-Nigerias-Equity-Fixed-Income-Market.png',
    'sitename': 'MarketForces Africa',
    'title': "Investment Returns Swerve in Nigeria's Equity, Fixed Income...",
    'url': 'https://dmarketforces.com/investment-returns-swerve-in-nigerias-equity-fixed-income-market/'}
{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/09/Oil-Prices-Jump-as-OPEC-Meet-to-Discuss-Output-Raise.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Oil Prices Jump as OPEC+ Meet to Discuss Output Raise

{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Valuation-Spots-Weak-Upside-as-Cost-Pressure-Dents-Earnings.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'NB Plc: Valuation Spots Weak Upside as Cost Pressure Dents '
             'Earnings',
    'url': 'https://dmarketforces.com/nb-plc-valuation-spots-weak-upside-as-cost-pressure-dents-earnings/'}
{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Okomu-Oil-Analysts-Project-20-Upside-For-Stock-See-Increase-Dividend.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Okomu Oil: Analysts

{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Ethiopia-Moves-to-Phase-out-Facebook-Twitter-Others.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Ethiopia Moves to Phase Out Facebook, Twitter, Others',
    'url': 'https://dmarketforces.com/ethiopia-moves-to-face-out-facebook-twitter-others/'}
{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Business-Owners-Dialogue-to-Reap-AfCFTA-Gains-Using-Digital-Platforms.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Business Owners Dialogue to Reap AfCFTA Gains Using Digital '
          

In [195]:
#Format Investment News table
inv_table = pd.DataFrame(dat)
inv_table.drop(["color", "favicon"], axis = 1)
inv_table = inv_table.reindex(columns = ["sitename", "description", "title", "url", "image"])
inv_table

,sitename,description,title,url,image
0,MarketForces Africa,Page not found - MarketForces Africa Recent post,"Investment Returns Swerve in Nigeria's Equity,...",https://dmarketforces.com/investment-returns-s...,https://dmarketforces.com/wp-content/uploads/2...
1,MarketForces Africa,Page not found - MarketForces Africa Recent post,Oil Prices Jump as OPEC+ Meet to Discuss Outpu...,https://dmarketforces.com/oil-prices-jump-as-o...,https://dmarketforces.com/wp-content/uploads/2...
2,MarketForces Africa,Page not found - MarketForces Africa Recent post,Federal Government Disburses N56 Billion for S...,https://dmarketforces.com/federal-government-d...,https://dmarketforces.com/wp-content/uploads/2...
3,MarketForces Africa,Page not found - MarketForces Africa Recent post,Investors See Lower Yield on T-Bills as Interb...,https://dmarketforces.com/investors-see-lower-...,https://dmarketforces.com/wp-content/uploads/2...
4,MarketForces Africa,Page not found - MarketForces Africa Recent post,Nigerian Exchange Sinks as Investors Sustain S...,https://dmarketforces.com/nigerian-exchange-si...,https://dmarketforces.com/wp-content/uploads/2...
5,MarketForces Africa,Page not found - MarketForces Africa Recent post,LoftyInc Capital Launched $10m Support for Afr...,https://dmarketforces.com/loftyinc-capital-lau...,https://dmarketforces.com/wp-content/uploads/2...
6,MarketForces Africa,Page not found - MarketForces Africa Recent post,"Agusto forecasts 13% NPL, says Banks exposures...",https://dmarketforces.com/agusto-forecasts-13-...,https://i2.wp.com/dmarketforces.com/wp-content...
7,MarketForces Africa,Page not found - MarketForces Africa Recent post,Growth-starved Nigeria's Economy Expends 83% o...,https://dmarketforces.com/growth-starved-niger...,https://dmarketforces.com/wp-content/uploads/2...
8,MarketForces Africa,Page not found - MarketForces Africa Recent post,The Reality and Fantasy of Being an Entrepreneur,https://dmarketforces.com/the-reality-and-fant...,https://dmarketforces.com/wp-content/uploads/2...
9,MarketForces Africa,Page not found - MarketForces Africa Recent post,"Investment Returns Swerve in Nigeria's Equity,...",https://dmarketforces.com/investment-returns-s...,https://dmarketforces.com/wp-content/uploads/2...


### Featured

In [198]:
# Call website (link)
site_link = "https://dmarketforces.com/category/investment/"
response  = requests.get(site_link)
response.status_code

#beautify content to make it readable
soup = bs(response.content, "html.parser")
#print(soup.prettify())

# inspect and Find all class of interest 
site_content = soup.find_all("a", class_= "td-image-wrap")
#site_content #td-image-wrap
    
# Collect all links in site content in a list
links = []
for item in site_content:
    link = item.attrs["href"]
    links.append(link)
#print(links)

# Get info for each link in Link List
dat = []
for link in links:
    lis = scrape_page_metadata(site_link, link)
    dat.append(lis)
#print(lis)
    

{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/09/Investment-Returns-Swerve-in-Nigerias-Equity-Fixed-Income-Market.png',
    'sitename': 'MarketForces Africa',
    'title': "Investment Returns Swerve in Nigeria's Equity, Fixed Income...",
    'url': 'https://dmarketforces.com/investment-returns-swerve-in-nigerias-equity-fixed-income-market/'}
{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/09/Oil-Prices-Jump-as-OPEC-Meet-to-Discuss-Output-Raise.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Oil Prices Jump as OPEC+ Meet to Discuss Output Raise

{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Valuation-Spots-Weak-Upside-as-Cost-Pressure-Dents-Earnings.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'NB Plc: Valuation Spots Weak Upside as Cost Pressure Dents '
             'Earnings',
    'url': 'https://dmarketforces.com/nb-plc-valuation-spots-weak-upside-as-cost-pressure-dents-earnings/'}
{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Okomu-Oil-Analysts-Project-20-Upside-For-Stock-See-Increase-Dividend.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Okomu Oil: Analysts

{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Ethiopia-Moves-to-Phase-out-Facebook-Twitter-Others.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Ethiopia Moves to Phase Out Facebook, Twitter, Others',
    'url': 'https://dmarketforces.com/ethiopia-moves-to-face-out-facebook-twitter-others/'}
{   'color': None,
    'description': 'Page not found - MarketForces Africa Recent post',
    'favicon': '//dmarketforces.com/wp-content/uploads/2020/04/cropped-MarketForces-e1602884604232-1-32x32.jpg',
    'image': 'https://dmarketforces.com/wp-content/uploads/2021/08/Business-Owners-Dialogue-to-Reap-AfCFTA-Gains-Using-Digital-Platforms.jpg',
    'sitename': 'MarketForces Africa',
    'title': 'Business Owners Dialogue to Reap AfCFTA Gains Using Digital '
          

In [200]:
#Format Featured table
feat_table = pd.DataFrame(dat)
feat_table.drop(["color", "favicon"], axis = 1)
feat_table = feat_table.reindex(columns = ["sitename", "description", "title", "url", "image"])
feat_table

,sitename,description,title,url,image
0,MarketForces Africa,Page not found - MarketForces Africa Recent post,"Investment Returns Swerve in Nigeria's Equity,...",https://dmarketforces.com/investment-returns-s...,https://dmarketforces.com/wp-content/uploads/2...
1,MarketForces Africa,Page not found - MarketForces Africa Recent post,Oil Prices Jump as OPEC+ Meet to Discuss Outpu...,https://dmarketforces.com/oil-prices-jump-as-o...,https://dmarketforces.com/wp-content/uploads/2...
2,MarketForces Africa,Page not found - MarketForces Africa Recent post,Federal Government Disburses N56 Billion for S...,https://dmarketforces.com/federal-government-d...,https://dmarketforces.com/wp-content/uploads/2...
3,MarketForces Africa,Page not found - MarketForces Africa Recent post,Investors See Lower Yield on T-Bills as Interb...,https://dmarketforces.com/investors-see-lower-...,https://dmarketforces.com/wp-content/uploads/2...
4,MarketForces Africa,Page not found - MarketForces Africa Recent post,Nigerian Exchange Sinks as Investors Sustain S...,https://dmarketforces.com/nigerian-exchange-si...,https://dmarketforces.com/wp-content/uploads/2...
5,MarketForces Africa,Page not found - MarketForces Africa Recent post,LoftyInc Capital Launched $10m Support for Afr...,https://dmarketforces.com/loftyinc-capital-lau...,https://dmarketforces.com/wp-content/uploads/2...
6,MarketForces Africa,Page not found - MarketForces Africa Recent post,"Agusto forecasts 13% NPL, says Banks exposures...",https://dmarketforces.com/agusto-forecasts-13-...,https://i2.wp.com/dmarketforces.com/wp-content...
7,MarketForces Africa,Page not found - MarketForces Africa Recent post,Growth-starved Nigeria's Economy Expends 83% o...,https://dmarketforces.com/growth-starved-niger...,https://dmarketforces.com/wp-content/uploads/2...
8,MarketForces Africa,Page not found - MarketForces Africa Recent post,The Reality and Fantasy of Being an Entrepreneur,https://dmarketforces.com/the-reality-and-fant...,https://dmarketforces.com/wp-content/uploads/2...
9,MarketForces Africa,Page not found - MarketForces Africa Recent post,"Investment Returns Swerve in Nigeria's Equity,...",https://dmarketforces.com/investment-returns-s...,https://dmarketforces.com/wp-content/uploads/2...


In [201]:
#save DataFrames to excel
writer = pd.ExcelWriter("dmarketforces_news.xlsx", engine = "xlsxwriter")

news_table.to_excel(writer, sheet_name = "News")
econews_table.to_excel(writer, sheet_name = "Economy News")
analy_table.to_excel(writer, sheet_name = "Analysis")
wb_table.to_excel(writer, sheet_name = "World Business News")
wi_table.to_excel(writer, sheet_name = "World Industries News")
hc_table.to_excel(writer, sheet_name = "Healthcare News")
inv_table.to_excel(writer, sheet_name = "Investment News")
feat_table.to_excel(writer, sheet_name = "Featured")

writer.save()




#### Ignore Below codes, rough work for further works
I Intend to write a reusable function code, but i'm tired. will finish up mater, Ignore the codes below.

In [173]:
from bs4 import BeautifulSoup as bs
# Query website using website url
def query_site(site_url:str):
    response  = requests.get(site_url)
    
    #beautify content to make it readable
    soup = bs(response.content, "html.parser")

    # inspect and Find all class of interest 
    site_content = (soup.find("ul", class_= "td-mobile-main-menu")).find_all("a")
    
    #create list of main navigation links in site
    global links
    links = []
    for item in site_content:
        link = item.attrs["href"]
        links.append(link)
    return links

def query_links(links:list):
    cat_links = links[1:]
    for i in range(len(cat_links)):
        response[i] = requests.get(cat_links[i])
        response_status = response[i].status_code
        
    #beautify content to make it readable
        soup[i] = bs(response[i].content, "html.parser")
    
    # inspect and Find all class of interest
        site_content[i] = soup[i].find_all("a", class_= "td-image-wrap")
    
    """"# Collect all links in site content in a list
        links2 = []
        for item in site_content[i]:
            link = item.attrs["href"]
            links2.append(link)
        
    # Get info for each link in Links List    
    global data
    site_url = links[0]
    data = []
    for i in links2:
        lis = scrape_page_metadata(site_url, link[i])
        data.append(lis)
        
    return(data)"""


def make_table(data):
    blog_posts = pd.DataFrame(data)
    blog_posts.drop(["color", "favicon"], axis = 1)
    blog_posts = finnews_info.reindex(columns = ["sitename",  "description", "title", "url", "image"])
    return blog_posts



In [111]:
#beautify content to make it readable
soup2 = bs(response2.content, "html.parser")
#print(soup.prettify())

# inspect and Find all class of interest 
site_content2 = (soup2.find("ul", class_= "td-mobile-main-menu")).find_all("a")
#site_content2 = list(site_content2)
site_content2 #td-image-wrap

[<a href="https://dmarketforces.com/">Home</a>,
 <a href="https://dmarketforces.com/category/news/">News</a>,
 <a href="https://dmarketforces.com/category/economy-news-archives/">Economy</a>,
 <a href="https://dmarketforces.com/category/analysis/">Analysis</a>,
 <a href="https://dmarketforces.com/category/world-news/business/">Banking &amp; Finance</a>,
 <a href="https://dmarketforces.com/category/world-news/industries/">Industries</a>,
 <a href="https://dmarketforces.com/category/news/healthcare/">Entrepreneurship</a>,
 <a href="https://dmarketforces.com/category/investments/">Investments</a>,
 <a href="https://dmarketforces.com/category/featured/">Feature</a>]

In [112]:
type(site_content2)

bs4.element.ResultSet

In [114]:
links2 = []
for item in site_content2:
    link = item.attrs["href"]
    links2.append(link)
    
print(links2)



['https://dmarketforces.com/', 'https://dmarketforces.com/category/news/', 'https://dmarketforces.com/category/economy-news-archives/', 'https://dmarketforces.com/category/analysis/', 'https://dmarketforces.com/category/world-news/business/', 'https://dmarketforces.com/category/world-news/industries/', 'https://dmarketforces.com/category/news/healthcare/', 'https://dmarketforces.com/category/investments/', 'https://dmarketforces.com/category/featured/']


In [ ]:
""""def generate_table(url):
        """Scrape target URL for metadata."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    pp = pprint.PrettyPrinter(indent=4)
    r = requests.get(url, headers=headers)
    html = bs(r.content, 'html.parser')
    metadata = {
        'title': get_title(html),
        'description': get_description(html),
        'image': get_image(html),
        'favicon': get_favicon(html, url),
        'sitename': get_site_name(html, url),
        'color': get_theme_color(html),
        'url': url
        }
    pp.pprint(metadata)
    return metadata
    
    
def beautify(response.content):
    soup = bs(response.content, "html.parser")
    return soup.prettify()

def find_class(soup):
    site_content = soup.find_all("a", class_= "td-image-wrap")
    return site_content

def get_links(data):
    for item in data:
        link = item.attrs["href"]
        return link

def list_link(link):
    links = []
    for item in site_content:
        link = item.attrs["href"]
        links.append(link)
    return links

def link_info(links):
    dat = []
    for link in links:
        lis = scrape_page_metadata(link)
        dat.append(lis)
    return lis

def format_table(dat):
    _info = pd.DataFrame(dat)
    _info.drop(["color", "favicon"], axis = 1)
    _info = news_info.reindex(columns = ["sitename", "description", "title", "url", "image"])
    return _info

    
    